In [2]:
# Imports
import pandas as pd
import seaborn as sns
import numpy as np
import networkx as nx
import csv
import visJS2jupyter.visualizations as visualizations
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from networkx.algorithms import community
from bokeh.models import EdgesAndLinkedNodes, NodesAndLinkedEdges
from networkx.algorithms.community import greedy_modularity_communities
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot

In [3]:
# cleaning dataframe
def clean_df(df):
  df.columns = df.columns.str.replace(r'_nonpart_part', '')
  df1 = df.copy()
  df1['index'] = df1.index
  contact_type_list = ['index', 'friend',	'workmate',	'neighbour',	'familymember',	'partner',	'colleague_association',	'other']
  df1 = df1[contact_type_list]
  df1 = df1.set_index('index').dot(df1.columns[1:] + ', ').str[:-2].reset_index(name='contact_type')
  df_all = pd.concat([df, df1], axis = 1)
  final_columns = ['sender_id','receiver_id','contact_type','frequency','contact_years','trust']
  final_df = df_all.assign(**{col : 'NA' for col in final_columns if col not in df_all.columns})[final_columns]
  final_df = final_df[['sender_id','receiver_id','contact_type','frequency','contact_years','trust']]
  final_df = final_df[final_df['sender_id'].notna()]
  final_df = final_df[final_df['receiver_id'].notna()]
  final_df['sender_id'] = final_df['sender_id'].astype(int)
  final_df['receiver_id'] = final_df['receiver_id'].astype(int)
  return final_df 

In [4]:
# agro participating farmers connection
part_part_df = pd.read_csv('data/Edges_agroconcept_complete.csv',sep=";")
part_part_df = clean_df(part_part_df)

part_part_df.head(2)

,sender_id,receiver_id,contact_type,frequency,contact_years,trust
0,1,2,colleague_association,1,NA,4
1,1,20,"workmate, colleague_association",2,NA,4


In [5]:
# agro participating farmers with external connections- the extrenal connection ID looks similar to farmers ID, have a find a way to filter them
part_others_df = pd.read_csv('data/Edges_agroconcept_and_external_contacts.csv',sep=";")
part_others_df = clean_df(part_others_df)
part_others_df.head(2)

,sender_id,receiver_id,contact_type,frequency,contact_years,trust
0,1,2,colleague_association,1,NA,4
1,1,29,other,2,NA,4


In [6]:
# non participating farmers with non participating farmers
non_non_df = pd.read_csv('data/Edges_nonpart_all.csv',sep=";")
non_non_df = clean_df(non_non_df)
non_non_df.head(2)

,sender_id,receiver_id,contact_type,frequency,contact_years,trust
0,57,82,"friend, workmate",3.0,NaN,4.0
1,57,83,workmate,2.0,NaN,4.0


In [7]:
# non participating farmers with agro participating farmers
non_part_df = pd.read_csv('data/Edges_nonpart_to_agroconcept.csv',sep=";")
non_part_df = clean_df(non_part_df)
non_part_df.head(2)


,sender_id,receiver_id,contact_type,frequency,contact_years,trust
0,57,21,workmate,1.0,10,NA
1,58,28,"workmate, other",2.0,50,NA


In [8]:
# grouping farmers and survey responders

# agro participating farmers ID
att_part_df = pd.read_csv('data/Atts_agroconcept_int.csv',sep=";")
part_list = att_part_df['ego_id'].tolist()

# agro participating farmers ID
att_non_df = pd.read_csv('data/Atts_nonpart_int.csv',sep=";")
non_list = att_non_df['ego_id_nonpart'].tolist()
print(part_list, non_list)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 26, 27, 28] [57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]


In [9]:
all_farmers_df = pd.concat([part_part_df, part_others_df, non_non_df, non_part_df])
all_farmers_df['participating'] = np.where( all_farmers_df['sender_id'].isin(part_list),'Yes'
, np.where((all_farmers_df['sender_id'].isin(non_list)) & (~all_farmers_df['sender_id'].isin(part_list)), 'No', 'Others' ))
# all_farmers_df.isna().sum() # checking if there are null values in sender_id or receiver_id columns
all_farmers_df = all_farmers_df.drop_duplicates(subset=['sender_id','receiver_id'])
all_farmers_df.head()

,sender_id,receiver_id,contact_type,frequency,contact_years,trust,participating
0,1,2,colleague_association,1.0,NA,4,Yes
1,1,20,"workmate, colleague_association",2.0,NA,4,Yes
2,1,26,"workmate, colleague_association",1.0,NA,4,Yes
3,1,14,"workmate, colleague_association",2.0,NA,4,Yes
4,1,15,"workmate, neighbour, colleague_association",2.0,NA,4,Yes


Reference for the below visualization: 
https://plotly.com/python/network-graphs/

In [11]:
sender_list = list(all_farmers_df["sender_id"].unique())
receiver_list = list(all_farmers_df["receiver_id"].unique())
node_list = set(sender_list + receiver_list)

G = nx.Graph()

for i in node_list:
    G.add_node(i)

for i,j in all_farmers_df.iterrows():
    G.add_edges_from([(j["sender_id"],j["receiver_id"])])

pos = nx.spring_layout(G, k=0.5, iterations=50)

for n, p in pos.items():
    G.nodes[n]['pos'] = p

edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='RdBu',
        reversescale=True,
        color=[],
        size=15,
        colorbar=dict(
            thickness=10,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=0)))

for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])

node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    # node_text.append('# of connections: '+str(len(adjacencies[1])))
    node_text.append('FarmerID: '+str(node))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text


fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>All Farmers and Contacts Network',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    # text="No. of connections",
                    showarrow=False,
                    xref="paper", yref="paper") ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

iplot(fig)
# fig.show()

We have used the reference from https://melaniewalsh.github.io/Intro-Cultural-Analytics/06-Network-Analysis/02-Making-Network-Viz-with-Bokeh.html#network-with-responsive-highlighting for the following visualizations. 

In [16]:
part_colour = dict.fromkeys(all_farmers_df.receiver_id.tolist() + all_farmers_df.sender_id.tolist())
for key, value in part_colour.items():
    # print(key, value)
    if key in part_list:
        part_colour[key] = '#a4d2d0' # green
    elif key in non_list:
        part_colour[key] = '#d2a4a6' # red
    else:
        part_colour[key] = '#e9e3ae' # yellow

In [17]:
G = nx.from_pandas_edgelist(all_farmers_df, 'sender_id', 'receiver_id')

# Calculate degree for each node and add as node attribute
degrees = dict(nx.degree(G))
nx.set_node_attributes(G, name='degree', values=degrees)

# Slightly adjust degree so that the nodes with very small degrees are still visible
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in nx.degree(G)])
nx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)

# Calculate communities
communities = greedy_modularity_communities(G)
communities = communities[:-1]

# Create empty dictionaries
modularity_class = {}
# modularity_color = {}
#Loop through each community in the network
for community_number, community in enumerate(communities):
    #For each member of the community, add their community number and a distinct color
    for name in community: 
        modularity_class[name] = community_number
        # modularity_color[name] = Spectral8[community_number]

# Add modularity class and color as attributes from the network above
nx.set_node_attributes(G, modularity_class, 'modularity_class')
nx.set_node_attributes(G, part_colour, 'part_colour')

#Choose colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'part_colour'

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8

#Choose a title!
title = 'All Farmers Network'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("Farmer ID", "@index"),
        ("Degree", "@degree"),
         ("Modularity Class", "@modularity_class"),
        ("Participating Color", "$color[swatch]:part_colour"),
]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(G, nx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as category from attribute)
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=color_by_this_attribute)
#Set node highlight colors
network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)
#Set edge highlight colors
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

#Highlight nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()

plot.renderers.append(network_graph)

show(plot)
# save(plot, filename=f"{title}.html")
output_notebook()

Loading BokehJS ...